# AIS Vessel Mobility Pattern Image Classification using SageMaker JumpStart Pre-Trained Model Endpoint
In this notebook, we will classify AIS Vessel Mobility pattern image using AWS SageMaker JumpStart trained model. 

### Download test images
(https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0)

In [5]:
import boto3
from IPython.core.display import HTML

region = boto3.Session().region_name
s3_bucket = f"eecse-6893-track-images"
key_prefix_m = "test/Moored"
key_prefix_a = "test/Anchored"
key_prefix_f = "test/Fishing"
key_prefix_u = "test/Underway"
s3 = boto3.client("s3")

def download_from_s3(key_prefix, key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)

moored_png = "IMO7618234-2015-09-13-2015-09-13.png" 
anchored_png = "IMO9263760-2015-09-11-2015-09-13.png"
fishing_png = "IMO7215654-2015-09-11-2015-09-13.png"
underway_png = "IMO7742293-2015-09-11-2015-09-13.png"

download_from_s3(key_prefix=key_prefix_m, key_filenames=[moored_png])
download_from_s3(key_prefix=key_prefix_a, key_filenames=[anchored_png])
download_from_s3(key_prefix=key_prefix_f, key_filenames=[fishing_png])
download_from_s3(key_prefix=key_prefix_u, key_filenames=[underway_png])

HTML('<table><tr><td> <img src="IMO7618234-2015-09-13-2015-09-13.png" alt="Moored" style="height: 250px;"/> <figcaption>Moored.png</figcaption>'
     '</td><td> <img src="IMO9263760-2015-09-11-2015-09-13.png" alt="Anchored" style="height: 250px;"/> <figcaption>Anchored.png</figcaption>'
     '</td><td> <img src="IMO7215654-2015-09-11-2015-09-13.png" alt="Fishing" style="height: 250px;"/> <figcaption>Fishing.png</figcaption>'
     '</td><td> <img src="IMO7742293-2015-09-11-2015-09-13.png" alt="Underway" style="height: 250px;"/> <figcaption>Underway.png</figcaption>'
     '</td>'
     '</tr></table>')

Moored.png,Anchored.png,Fishing.png,Underway.png


### Open the downloaded images and load in memory

In [6]:
images = {}
with open(moored_png, 'rb') as file: images[moored_png] = file.read()
with open(anchored_png, 'rb') as file: images[anchored_png] = file.read()
with open(fishing_png, 'rb') as file: images[fishing_png] = file.read()
with open(underway_png, 'rb') as file: images[underway_png] = file.read()

### Query endpoint created from SageMaker JumpStart (MobileNetV2) after training the model using our training data
We trained MobileNetV2 pre-trained (ImageNet dataset trained) model that is trained on our training data and deployed as SageMaket endpoint using AWS SageMaker JumpStart studio.

In [7]:
import json
image_labels = [ 'Anchored', 'Fishing', 'Moored', 'Underway' ]

def query_endpoint(img):
    endpoint_name = 'jumpstart-ftc-tf-ic-tf2-preview-mobilenet-v2-classific'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                      ContentType='application/x-image', 
                                      Body=img)
    model_predictions = json.loads(response['Body'].read())['predictions'][0]
    return model_predictions

for filename, img in images.items():
    model_predictions = query_endpoint(img)  
    top4_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:4]
    top4_class_labels = ", ".join([image_labels[id] for id in top4_prediction_ids])
    display(HTML(f'<img src="{filename}" alt={filename} align="left" style="width: 250px;"/>' 
                 f'<figcaption>Model predictions (in order) are: {top4_class_labels}</figcaption>'))

### Query endpoint created from SageMaker JumpStart (EfficientNetB0) after training the model using our training data
We trained EfficientNetB0 pre-trained (ImageNet dataset trained) model that is trained on our training data and deployed as SageMaket endpoint using AWS SageMaker JumpStart studio.

In [10]:
import json
image_labels = [ 'Anchored', 'Fishing', 'Moored', 'Underway' ]

def query_endpoint(img):
    endpoint_name = 'jumpstart-ftc-tf-ic-efficientnet-b0-classification-1'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                      ContentType='application/x-image', 
                                      Body=img)
    model_predictions = json.loads(response['Body'].read())['predictions'][0]
    return model_predictions

for filename, img in images.items():
    model_predictions = query_endpoint(img)  
    top4_prediction_ids = sorted(range(len(model_predictions)), key=lambda index: model_predictions[index], reverse=True)[:4]
    top4_class_labels = ", ".join([image_labels[id] for id in top4_prediction_ids])
    display(HTML(f'<img src="{filename}" alt={filename} align="center" style="width: 250px;"/>' 
                 f'<figcaption align="center">Model predictions (in order) are: {top4_class_labels}</figcaption>'))